# Simulation Credit Example

**[Optional for Google Colab] Installing PiML**

1. Run `!pip install piml` to install the latest version of PiML
2. In Colab, you'll need restart the runtime in order to use newly installed PiML version.

In [ ]:
! pip install piml

## Data Description

This example demonstrates the use of PiML for fairness testing. We first simulate a credit decisioning data with hypothesized features `Mortgage`, `Balance`, `Amount Past Due`, `Delinquency status`, `Credit Inquiry`, `Open Trade`, `Utilization`, as well as demographic features `Gender` and `Race`. The response `Status` is a binary indicator, and this is a classification problem. 

**Sample Size**: 60526, **Columns Num**: 10. 

**Features:**

`Mortgage` *(Numerical)*: Applicant’s mortgage size.

`Balance` *(Numerical)*: Average last 12 months credit card balance.

`Amount Past Due` *(Numerical)*: The minimum required payment that was not applied to the account as of the last payment due date.

`Delinquency Status` *(Ordinal)*: 0: current, 1: < 30 day delinquent, 2: 30-60 day delinquent, 3: 60-90 day and so on.

`Credit Inquiry` *(Ordinal)*: Number of credit inquiries in the last 12 months.

`Open Trade` *(Ordinal)*: Number of open credit accounts. 

`Utilization` *(Numerical)*: % credit utilization, the sum of all your balances, divided by the sum of your cards' credit limits.

**Demographic Features:**

Demographic features in Credit data cannot be used for modeling.

`Gender` *(Categorical)*: Two kinds of gender.

`Race` *(Categorical)*: Two kinds of race. 

**Target Response:**

`Status` *(Categorical)*: 0: default (should not be approved) and 1: non-default (should be approved). The 0/1 ratio is nearly 1:5. 

## Load and Prepare data

In [2]:
from piml import Experiment
exp = Experiment()

# Manually load data to piml
import pandas as pd
data = pd.read_csv('https://github.com/SelfExplainML/PiML-Toolbox/blob/main/datasets/CreditSimuUnbalanced.csv?raw=true')
exp.data_loader(data=data)

,Mortgage,Balance,Amount Past Due,Delinquency Status,Credit Inquiry,Open Trade,Utilization,Gender,Race,Status
0,292916.21,979.20,0.00,0.0,0.0,0.0,0.629327,0.0,1.0,0.0
1,305767.40,2398.64,28.36,1.0,0.0,0.0,0.804077,1.0,1.0,0.0
2,257664.96,1830.87,1089.69,4.0,1.0,1.0,0.263113,1.0,1.0,0.0
3,152783.10,1524.16,948.84,5.0,3.0,0.0,0.390256,1.0,0.0,0.0
4,299243.25,2126.39,279.13,3.0,1.0,1.0,0.553556,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
60521,243584.72,1194.92,0.00,0.0,0.0,0.0,0.553532,0.0,1.0,1.0
60522,272959.22,856.45,0.00,0.0,1.0,1.0,0.678293,0.0,1.0,1.0
60523,225381.40,1084.89,0.00,0.0,0.0,0.0,0.631209,0.0,1.0,1.0
60524,154846.08,445.45,0.00,0.0,0.0,0.0,0.333873,1.0,1.0,1.0


In [3]:
# Exclude features one-by-one: "Gender", "Race" (demographic variables); 
exp.data_summary()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

HTML(value='<link rel="stylesheet" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.…

In [4]:
# Prepare dataset with Test Ratio = 0.2
exp.data_prepare()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

In [5]:
#Feature selection with RCIT method
exp.feature_select()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

## Model Train

Fit XGB2 and XGB2 with monotonicity.  
Monotonic increasing list: ['Mortgage', 'Balance', 'Amount Past Due'],   
Monotonic decreasing list: ['Utilization', 'Delinquency Status', 'Credit Inquiry', 'Open Trade'].

In [6]:
exp.model_train()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

## Fairness Testing

**Guide Step:**  

1. First select a registered model (in this case, XGB2_monotonic)  


2. Group Setting  
    - Set Add Category = "Gender", select "1.0" as reference, select "0.0" as protected, then click "Add".  
    - Set Add Category = "Race",  select "1.0" as reference, select "0.0" as protected, then click "Add".   
    
    
3. Debiasing/unfairness mitigation by Feature Binning
   - 3.1 Select AIR as fairness metric and F1 as performance metric.  
   - 3.2 Select 'Balance' as binning attribute, 'Customize' as binning method and binning range is 0-0.25.  
   - 3.3 Click the button "APPLY" to apply the binning setting to the data.   
   - 3.4 Click 'Evaluate' to compare the fairness and performance metrics after binning.  
   
   - If you want to add other binning configs to compare.
       - Click button "NEW CONFIG" to generate a new binning settting, after that repeat the 3.2-3.4 again to compare results between diffrent binning settings.
       - Click button "CLEAR ALL" could remove all the record.
   
   For this model, after binning the AIR of Gender improves to near 1 and the performance does not sacrifice too much.  
   
   
4. Debiasing/unfairness mitigation by Feature Removal
   - Select AIR as fairness metric and F1 as performance metric.  
   - Check the fairness and performance metrics upon removal of one feature at a time.  

    After removing Mortgage/Balance, the AIR of Race/Gender is increased near to 1.

In [7]:
exp.model_fairness()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

## Diagnose Testing  

**Guide Step:**


1. First select a registered model (in this case, XGB2_monotonic)  


2. Weakspot
    - Select `Mortgage` and `Delinquency Status` as 'Feature 1', respectively. Choose 'AUC' as metric and set threshold to 1.0.  
    - Check the weak area for each config.   
    
    For this model, the low mortgage and low deliquency status areas are both weak.
    
    
3. Reliability
   - Select `Mortgage` and `Utilization` as 'Feature', respectively.  
   - Check the uncertain area for each config.  
   
   For this model, the low mortgage and high utilization areas are both of high uncertainty.
   
   
4. Resilience
   - Select `Mortgage` and `Delinquency Status` as 'Plot Feature', respectively. Choose 'AUC' as metric.
   - Check the distribution shift of the 20% worst samples. 

    For this model, the low mortgage and low deliquency status areas have more worst samples than the rest.

In [8]:
exp.model_diagnose()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

## Model Comparing

**Guide Step:**

1. First select registered models (in this case, XGB2 and XGB2_monotonic)   

2. Robustness  
    - Choose 'AUC' as metric.
    - Check the model performance with perturbation on all features.   
    
    For these two models, the Monotonic XGB2 is more robust than XGB2.

In [9]:
exp.model_compare()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>